In [ ]:
from bs4 import BeautifulSoup
import requests
from markdownify import markdownify as md
from urllib.parse import urljoin

In [254]:
def get_pages(url):
    base_url = 'https://eduwiki.innopolis.university/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    lis = soup.find('ul', class_="mw-allpages-chunk").findAll('a', href=True)
    pages = []
    for li in lis:
        abs_url = urljoin(base_url, li['href'])
        pages.append(abs_url)
    return pages

In [256]:
pages = get_pages('https://eduwiki.innopolis.university/index.php/Special:AllPages')

In [265]:
pages

['https://eduwiki.innopolis.university/index.php/ALL:StudyPlan',
 'https://eduwiki.innopolis.university/index.php/About_lecturing_a_course',
 'https://eduwiki.innopolis.university/index.php/About_this_document',
 'https://eduwiki.innopolis.university/index.php/AcademicCalendar',
 'https://eduwiki.innopolis.university/index.php/AcademicLeave',
 'https://eduwiki.innopolis.university/index.php/Academic_Leave',
 'https://eduwiki.innopolis.university/index.php/AdilKhan',
 'https://eduwiki.innopolis.university/index.php/AhsanKazmi',
 'https://eduwiki.innopolis.university/index.php/AleksandrMaloletov',
 'https://eduwiki.innopolis.university/index.php/AlexanderHramov',
 'https://eduwiki.innopolis.university/index.php/AlexanderKlimchik',
 'https://eduwiki.innopolis.university/index.php/AlexanderTormasov',
 'https://eduwiki.innopolis.university/index.php/All:Schedule',
 'https://eduwiki.innopolis.university/index.php/All:Schedule_Google',
 'https://eduwiki.innopolis.university/index.php/AlmaOrac

In [259]:
def parse_pages(pages):
    for page_url in pages:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Decompose the 'div' with class 'printfooter'
        printfooter = soup.find('div', class_='printfooter')
        if printfooter:
            printfooter.decompose()

        # Decompose single elements: script, nav, footer tags
        for s in soup.findAll(['script', 'nav', 'footer']):
            s.decompose()

        # Decompose the 'div' with id 'mw-navigation'
        mw_navigation = soup.find('div', id='mw-navigation')
        if mw_navigation:
            mw_navigation.decompose()

        # Decompose the 'div' with id 'siteSub'
        site_sub = soup.find('div', id='siteSub')
        if site_sub:
            site_sub.decompose()

        # Decompose all 'a' tags with class 'mw-jump-link'
        for jump_link in soup.find_all('a', class_='mw-jump-link'):
            jump_link.decompose()


        # Process the body tag
        body_tag = soup.find('body')
        if body_tag:
            filename = page_url.replace('https://eduwiki.innopolis.university/', '').replace('index.php/', '').replace('/', '_') + '.md'
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(md(body_tag.decode_contents()))
                print(f"Markdown content saved to: {filename}")
        else:
            print("Body tag not found in the HTML content.")

P.S. Многие из этих ссылок какие-то ебанутые и хз зачем они существуют.
===
1. - https://eduwiki.innopolis.university/index.php/BSc:AcademicResearchandWritingCulture
  - https://eduwiki.innopolis.university/index.php/BSc:AcademicResearchandWritingCulture.previous_version
2. - https://eduwiki.innopolis.university/index.php/BSc:AcademicResearchandWritingCulture2
  - https://eduwiki.innopolis.university/index.php/BSc:AcademicResearchandWritingCulture2.S22

3. - https://eduwiki.innopolis.university/index.php/BSc:AnalyticGeometryAndLinearAlgebraII
 - https://eduwiki.innopolis.university/index.php/BSc:AnalyticGeometryAndLinearAlgebraII.S22
 - https://eduwiki.innopolis.university/index.php/BSc:AnalyticGeometryAndLinearAlgebraII_old
 - https://eduwiki.innopolis.university/index.php/BSc:AnalyticGeometryAndLinearAlgebraI_new
 - https://eduwiki.innopolis.university/index.php/BSc:AnalyticGeometryAndLinearAlgebraI_old
 - https://eduwiki.innopolis.university/index.php/BSc:AnalyticalGeometryAndLinearAlgebraI-CS

In [260]:
parse_pages(pages)

Markdown content saved to: ALL:StudyPlan.md
Markdown content saved to: About_lecturing_a_course.md
Markdown content saved to: About_this_document.md
Markdown content saved to: AcademicCalendar.md
Markdown content saved to: AcademicLeave.md
Markdown content saved to: Academic_Leave.md
Markdown content saved to: AdilKhan.md
Markdown content saved to: AhsanKazmi.md
Markdown content saved to: AleksandrMaloletov.md
Markdown content saved to: AlexanderHramov.md
Markdown content saved to: AlexanderKlimchik.md
Markdown content saved to: AlexanderTormasov.md
Markdown content saved to: All:Schedule.md
Markdown content saved to: All:Schedule_Google.md
Markdown content saved to: AlmaOracevic.md
Markdown content saved to: AnastasiyaPuzankova.md
Markdown content saved to: AndreyFrolov.md
Markdown content saved to: AndreySadovykh.md
Markdown content saved to: ArtemBurmyakov.md
Markdown content saved to: ArtemKruglov.md
Markdown content saved to: BS1.md
Markdown content saved to: BS1:CS:StudyPlan.md
M

In [264]:
!cat /content/BSc:IntroductionToProgramming.md








BSc: Introduction To Programming



(Redirected from [BSc:IntroductionToProgramming](/index.php?title=BSc:IntroductionToProgramming&redirect=no "BSc:IntroductionToProgramming"))


Contents
--------


* [1 Introduction to Programming](#Introduction_to_Programming)
	+ [1.1 Short Description](#Short_Description)
	+ [1.2 Prerequisites](#Prerequisites)
		- [1.2.1 Prerequisite subjects](#Prerequisite_subjects)
		- [1.2.2 Prerequisite topics](#Prerequisite_topics)
	+ [1.3 Course Topics](#Course_Topics)
	+ [1.4 Intended Learning Outcomes (ILOs)](#Intended_Learning_Outcomes_.28ILOs.29)
		- [1.4.1 What is the main purpose of this course?](#What_is_the_main_purpose_of_this_course.3F)
		- [1.4.2 ILOs defined at three levels](#ILOs_defined_at_three_levels)
			* [1.4.2.1 Level 1: What concepts should a student know/remember/explain?](#Level_1:_What_concepts_should_a_student_know.2Fremember.2Fexplain.3F)
			* [1.4.2.2 Level 2: What basic practical skills should a student be able to perform?](#

In [244]:
# !rm -r /content/*.md